In [7]:
%pip install networkx

Note: you may need to restart the kernel to use updated packages.


# Graph + Warshall

In [4]:
import pandas as pd
import networkx as nx
import numpy as np

In [54]:
routes = pd.read_excel('data/Distance.xlsx', sheet_name='Roads', index_col='road_id')

In [19]:
def get_node(route):
    travel_time = ((route.distance / 1000) / 30) * 60
    return tuple([route.source_point_id, route.target_point_id, {'weight': travel_time}])

In [20]:
edges = list(routes.apply(get_node, axis=1).values)

In [21]:
graph = nx.DiGraph(edges)

In [22]:
predecessor, distances = nx.floyd_warshall_predecessor_and_distance(graph)

In [23]:
all_points = list(set(routes.source_point_id) | set(routes.target_point_id))

In [24]:
warshall_map = {}

for source_point in all_points:
    warshall_map[source_point] = {}
    warshall_map[source_point]['distance'] = {}
    warshall_map[source_point]['route'] = {}
    
    for destination_point in all_points:
        warshall_map[source_point]['distance'][destination_point] = distances[source_point][destination_point]
        warshall_map[source_point]['route'][destination_point] = nx.reconstruct_path(source_point, destination_point, predecessor)

# Vodila Generator

In [15]:
routes = pd.read_excel('data/Distance.xlsx', sheet_name='Roads', index_col='road_id')

In [16]:
all_points = list(set(routes.source_point_id) | set(routes.target_point_id))

In [118]:
default_spawnpoint_id = 4
bus_capacities = [100] * 30 + [50] * 10

In [65]:
def send_awaited_model_time_stub(next_model_time: float):
    pass

In [129]:
def simulate_synchronizer_response(global_model_time) -> float:
    return global_model_time + np.random.random(1)[0]

In [130]:
def get_destination(points: list, curr_location: int) -> int:
    destination = None
    while True:
        destination = np.random.choice(points, size=1)[0]
        if destination == curr_location:
            continue
        break
    return destination

In [131]:
global_model_time = 0

def wake_yarik(warshall_map: dict):
    default_spawnpoint_id = 4
    bus_capacities = [100] * 30 + [50] * 10
    
    routes = pd.read_excel('data/Distance.xlsx', sheet_name='Roads', index_col='road_id')
    all_points = list(set(routes.source_point_id) | set(routes.target_point_id))
    
    drivers = []
    for driver_id, bus_capacity in enumerate(bus_capacities):
        drivers.append({
            'id': driver_id,
            'location': default_spawnpoint_id,
            'destination': None,
            # 'task_id': None,
            'capacity': bus_capacity,
            'free_time': None
        })

    future_events_chain = [0] * 40
    
    return all_points, drivers, future_events_chain

In [ ]:
def assign_task():
    pass

In [ ]:
# DRIVERS INIT CYCLE

for driver in drivers:
    # Get current driver location
    location = driver['location']
    # Get random destination, assign to the driver
    destination = get_destination(all_points, location)
    driver['destination'] = destination

    # Get next route point
    next_point = warshall_map[location]['route'][destination][1]
    # Get time until the next point, update future events chain
    model_timedelta = warshall_map[location]['distance'][next_point]
    future_events_chain[driver['id']] = (driver['id'], model_timedelta)

    # Get full time until destination point, update driver's time until being free
    free_timedelta = warshall_map[location]['distance'][destination]
    driver['free_time'] = free_timedelta

In [132]:
# Get closest model time to an event, send it to the synchronizer
future_events_chain = sorted(future_events_chain, key=lambda tup: tup[1])
send_awaited_model_time_stub(future_events_chain[0][1])

In [134]:
# Get new model time
old_global_model_time = global_model_time
global_model_time = simulate_synchronizer_response(global_model_time)

# Get time passed
global_timedelta = global_model_time - old_global_model_time

In [156]:
def move_drivers(
    drivers: list,
    future_events_chain: list,
    global_model_time: float,
    warshall_map: dict
) -> list:
    # Update drivers position according to the time passed
    for driver_id, event_model_time in future_events_chain:
        # If the model time moved enough to trigger a driver's event
        if global_model_time >= event_model_time:
            # Get destination, old_location and new location
            destination = driver['destination']
            old_location = driver['location']
            location = warshall_map[old_location]['route'][destination][1]

            # Get time travelled to the new point
            travel_time = warshall_map[location]['distance'][old_location]

            # If the driver reached his destination
            if location == destination:
                drivers[driver_id]['location'] = destination
                # drivers[driver_id]['task_id'] = None
                drivers[driver_id]['destination'] = None
                drivers[driver_id]['free_time'] = None
                continue

            drivers[driver_id]['location'] = location
            drivers[driver_id]['free_time'] -= travel_time
            
    return drivers

In [ ]:
# # Update drivers position according to the time passed
# for driver_id, event_model_time in future_events_chain:
#     # If the model time moved enough to trigger a driver's event
#     if global_model_time >= event_model_time:
#         # Get destination, old_location and new location
#         destination = driver['destination']
#         old_location = driver['location']
#         location = warshall_map[old_location]['route'][destination][1]

#         # Get time travelled to the new point
#         travel_time = warshall_map[location]['distance'][old_location]

#         # If the driver reached his destination
#         if location == destination:
#             drivers[driver_id]['location'] = destination
#             # drivers[driver_id]['task_id'] = None
#             drivers[driver_id]['destination'] = None
#             drivers[driver_id]['free_time'] = None
#             continue

#         # Get local timedelta (amount of time for current driver to move around)
#         local_timedelta = global_timedelta - travel_time
#         # While driver has time to move along his path
#         while True:
#             # Get current location, next point and travel time needed to reach the next point
#             drivers[driver_id]['location'] = location
#             next_point = warshall_map[location]['route'][destination][1]
#             travel_time = warshall_map[location]['distance'][next_point]

#             # If the driver can reach the next point
#             if travel_time <= local_timedelta:
#                 # If the driver has arrived to his destination, set him free
#                 if next_point == destination:
#                     drivers[driver_id]['location'] = destination
#                     # drivers[driver_id]['task_id'] = None
#                     drivers[driver_id]['destination'] = None
#                     drivers[driver_id]['free_time'] = None
#                     break

#                 # Update the driver's location (has not reached the end point yeat, but still has time to move to the next point)
#                 drivers[driver_id]['location'] = next_point
#                 drivers[driver_id]['free_time'] -= travel_time
#                 local_timedelta -= travel_time
#             # If there's no time left to move to the next point, get the next driver id
#             else:
#                 break
# return drivers

In [ ]:
def update_future_events_chain(future_events_chain: list, drivers: list) -> list:
    for driver in drivers:
        # Get current driver location, destination and the next route point
        location = driver['location']
        destination = driver['destination']
        next_point = warshall_map[location]['route'][destination][1]
        
        # Get time until the next point, update future events chain
        event_model_time = warshall_map[location]['distance'][next_point] + global_model_time
        future_events_chain[driver['id']] = (driver['id'], event_model_time)

    return sorted(future_events_chain, key=lambda tup: tup[1])

# Yarik Workflow

1. Construct graph (get_warshall_mapping)
2. Get Warshall Map (get_warshall_mapping)
3. Wake up Yarik (wake_yarik)
4. Await task ({some_task_await_routine})
5. Assign task (assign_task)
6. Send model time change request (send_awaited_model_time)
7. Await response ({some_response_awaiting_routine})
8. Get new model time (get_synchronizer_response)
9. Update driver positions and future events chain (move_drivers -> update_future_events_chain)
10. Go to 6